In [2]:
import requests
import lxml.html as lh
import pandas as pd
import numpy as np

### Scrape wikipedia table

In [3]:
website_url=requests.get('http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

from bs4 import BeautifulSoup
soup=BeautifulSoup(website_url,'lxml') # or use html.parser
# print(soup.prettify())

In [4]:
My_table = soup.find('table',{'class':'wikitable sortable'}).tbody

In [5]:
#My_table

### Create a dataframe

In [6]:
PC=[]
B=[]
NH=[]

for row in My_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        PC.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        NH.append(cells[2].find(text=True))
        
import pandas as pd
df=pd.DataFrame(PC,columns=['PostCode'])
df['Borough']=B
df['Neighborhood']=NH
df.head()

,PostCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Group by and remove duplicates

In [7]:
#df.reset_index(inplace=True)
gb = df.groupby(['PostCode','Borough'])
gb

In [8]:
result = gb['Neighborhood'].unique()
#result

In [9]:
df2=pd.DataFrame(result)
df2.reset_index(inplace=True) 
df2.head()

,PostCode,Borough,Neighborhood
0,M1A,Not assigned,[Not assigned ]
1,M1B,Scarborough,"[Rouge, Malvern]"
2,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]"
3,M1E,Scarborough,"[Guildwood , Morningside, West Hill]"
4,M1G,Scarborough,[Woburn]


### Clean Neighborhood column for the brackets

In [10]:
def unfold(obj):
    toReturn = []
    for row in obj:
        strHolder = ""
        for x in row:
            if np.where(row == x)[0] == 0:
                strHolder = strHolder + x.replace('\n', '')
            else:
                strHolder = strHolder + ', ' + x.replace('\n', '')
        toReturn.append(strHolder)
    return toReturn



In [11]:
#print(np.array2string(df2[0,2].replace(r"[",'').replace(r"]",'')))
df2['Neighborhood']=  unfold(df2['Neighborhood'])
df2.head()
# unfold(df2['Neighborhood'])

,PostCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M1B,Scarborough,"Rouge, Malvern"
2,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
3,M1E,Scarborough,"Guildwood, Morningside, West Hill"
4,M1G,Scarborough,Woburn


### Import geocoordinates from file in cloudstorage and join with exiting dataframe.

In [12]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0


### hidden credentials (cell not shown)

In [40]:
# The code was removed by Watson Studio for sharing.

In [15]:
body = client_3ab5b2fa13d04b88b2cb7fab7b7e3670.get_object(Bucket='canadaproject-donotdelete-pr-xbsssamwskkalv',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)

In [16]:
df_data_1.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [17]:
df_loc = pd.merge(left=df2,right=df_data_1, how='left', left_on='PostCode', right_on='Postal Code')
df_loc.head()

,PostCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M1A,Not assigned,Not assigned,NaN,NaN,NaN
1,M1B,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
2,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
3,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
4,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917


In [18]:
df_loc1 = df_loc.dropna()
df_loc1.head()

,PostCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
1,M1B,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
2,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
3,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
4,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
5,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


In [24]:
df_loc2=df_loc1.drop(['Postal Code'], axis=1)
print (df_loc2.shape)
df_loc2.head()

(103, 5)


,PostCode,Borough,Neighborhood,Latitude,Longitude
1,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
2,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
3,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
4,M1G,Scarborough,Woburn,43.770992,-79.216917
5,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [20]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium
from folium import plugins
print('Folium installed and imported!')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.3.1               |             py_0          25 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    altair-4.0.0               |             py_0         606 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.1 MB

The following NEW packages will be 

In [39]:
# define the world map centered around Canada with a low zoom level
Toronto_map = folium.Map(location=[43.6532, -79.3832], zoom_start=12)

# display world map
Toronto_map

# instantiate a feature group for the incidents in the dataframe
loc2 = folium.map.FeatureGroup()

# loop through the 100 crimes and add each to the incidents feature group
for lat, lng, in zip(df_loc2['Latitude'], df_loc2['Longitude']):
    loc2.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='blue',
            fill=True,
            fill_color='red',
            fill_opacity=0.5
        )
    )

# add incidents to map
Toronto_map.add_child(loc2)